# Smart Contracts
Smart contracts are ***classes*** on the blockchain from which ***objects*** such as new tokens can be instantiated.  
A few facts about ***smart contracts***:
> Once compiled, smart contracts are executed by the Ethereum virtual machine - think of it as a single large computer on the blockchain       
> They are stored on the blockchain thorugh a transaction executed by an account, a real user or another contract. Fof course, the account must have sufficient funds to pay for the deployment fees      
> They are owned (and they remember) the account that deployed them but have their own address (there is no private key associated, though)  
> Just like everything else on the blockchain, they are immutable. Once deployed, the code cannot be changed; new versions of the same contract are issued as new contracts  
> They can only access data on the blockchain; if external data is needed (e.g. prices of real assets) it can be pushed to the smart contract from the outside world
> They are written in Solidity and compiled to bytecode, which is then deployed tp the Ethereum Virtual Machine.  

On a separate tab, open Remix (https://remix.ethereum.org/), a compiler for the Solidity langage.  
Click on the ***New File*** light blue button and choose a name for your smart contract.  Press enter and Remix will add ".sol", the extension used for Solidity source files.  

Start your code by telling the system which version of the compiler to use. You can use a recent one, like the 0.8 version.  

```Solidity
pragma solidity ^0.8.0;
```

You can now compile the code to check for errors.
In remix, navigate to the "Solidity compiler" icon, and click the compile button.

![Remix Compiler](https://remix-ide.readthedocs.io/en/latest/_images/a-icons-at-load.png)

The compiler will warn you that no license identifier was provided. You can add a license identifier or ignore the warning.  

```Solidity
//SPDX-License-Identifier: UNLICENSED
```

Note the two slashes (//) which can also be used to tell the compiler that what follows is a comment (same as # in python).  
The compiler will also report "No Contract Compiled Yet"  

We will add a contract (a class) that represents some sort of currency.    

```Solidity
contract Money {
    uint8 public constant decimals = 18;
    uint256 public totalSupply;
    string public name;
    string public symbol;
}
```
The syntax is different from Python, but the logic is similar:   
- instead of using the keyword ***class*** we use ***contract*** (afterall Solidity is not a general purpose language but a ***Smart Contract*** language    
- we specify the type of the attributes   
-- ***uint8*** stands for ***u***nsigned ***int***eger with a lenghth of ***8*** bit, i.e. a maximum of two at the power of eight, or 256 digits including the zero, therefore supporting numbers no greater than 255)  
-- ***unit256*** accomodates a much larger number (78 decimal digits), the total supply that the token will have at any given point in time.  Note that this may vary during the life of the contract, unlike the number of decimals which is ***constant*** and set to 18  
-- ***string*** represents a collection of characters  

If you compile the code, Remix will now recognise a contract and give you the option to publish it on various media provided by the blockchain.  
We will not use this option as we have not connected a wallet that has sufficient ethers to pay for the fees required to deploy the contracts.  

We now need to add a constructor to tell the system how we intend to instantiate objects from this contract.  As you probably guessed we will instantiate a token which we will later use on the blockchain.  
If you continue to follow the standard proposed in this tutorial (i.e. naming conventions, number of decimals, standard methods and attributes), you will end up with an ERC20 compliant token.

We add the constructor with the keyword ***constructor***:  

```Solidity
constructor(uint256 total, string memory tokenName, string memory tokenSymbol) {
        totalSupply = total;
        name = tokenName;
        symbol = tokenSymbol;
```

Remember to include the constructor method within the curly brackets of the Money contract (the constructor is afterall one of the methods of the class):    
```Solidity
//SPDX-License-Identifier: UNLICENSED
pragma solidity ^0.8.0;

contract Money {
    uint8 public constant decimals = 18;
    uint256 public totalSupply;
    string public name;
    string public symbol;

    constructor(uint256 total, string memory tokenName, string memory tokenSymbol) {
        totalSupply = total;
        name = tokenName;
        symbol = tokenSymbol;
    }
}
```

The account that will deploy this contract (and instantiate the token) will decide the key features of the token, by providing a name, a symbol and the total supply.  


#Cell 1

In [ ]:
print("Generating functions...") 

%pip install web3 -q

from eth_account import Account
from web3 import Web3, HTTPProvider
from web3.gas_strategies.rpc import rpc_gas_price_strategy
import json

def create_transaction(account_from, address_to, value, gas_price):
#    w3 = Web3(HTTPProvider('https://sepolia.infura.io/v3/fae476cc30ff43e0813948aca32f409d'))
    w3 = Web3(HTTPProvider('https://endpoints.omniatech.io/v1/eth/sepolia/public'))
    return dict (nonce=w3.eth.get_transaction_count(account_from),
            #maxFeePerGas = w3.to_wei(gas_price, 'gwei'),
            gas = 21000,
            gasPrice = w3.to_wei(gas_price, 'gwei'),
            to = address_to,
            chainId = 11155111,
            value = w3.to_wei(value, "ether"))

def sign_transaction(transaction, private_key):
    w3 = Web3(HTTPProvider('https://sepolia.infura.io/v3/fae476cc30ff43e0813948aca32f409d'))
    signed = w3.eth.account.sign_transaction(transaction, private_key)
    return signed.rawTransaction

def post_transaction(signed_transaction):
    w3 = Web3(HTTPProvider('https://sepolia.infura.io/v3/fae476cc30ff43e0813948aca32f409d'))
    tx_hash=w3.eth.send_raw_transaction(signed_transaction)
    return tx_hash

def create_contract_transaction(ABI, Bytecode, gas_price):
    w3 = Web3(HTTPProvider('https://sepolia.infura.io/v3/fae476cc30ff43e0813948aca32f409d'))
    contract_ = w3.eth.contract(abi=ABI, bytecode=Bytecode)
    txn = contract_.constructor().buildTransaction({
    'from': account_from,
    'nonce': w3.eth.get_transaction_count(account_from),
    'gas': 3000000,
    'gasPrice': w3.toWei(gas_price, 'gwei'),
    'chainId': 11155111})
    return txn

def load_abi(file):
    f = open(file)
    # a dictionary
    data = json.load(f)
    f.close()
    return data['abi']   

print("Functions generated, you can now use the functions on this workbook to derive Ethereum addresses from a given private key and transfer funds.")

#Cell 2

In [ ]:

byte = '60806040523480156200001157600080fd5b50604051620010bf380380620010bf83398181016040528101906200003791906200027c565b82600181905550826000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002081905550816002908162000092919062000557565b508060039081620000a4919062000557565b505050506200063e565b6000604051905090565b600080fd5b600080fd5b6000819050919050565b620000d781620000c2565b8114620000e357600080fd5b50565b600081519050620000f781620000cc565b92915050565b600080fd5b600080fd5b6000601f19601f8301169050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b620001528262000107565b810181811067ffffffffffffffff8211171562000174576200017362000118565b5b80604052505050565b600062000189620000ae565b905062000197828262000147565b919050565b600067ffffffffffffffff821115620001ba57620001b962000118565b5b620001c58262000107565b9050602081019050919050565b60005b83811015620001f2578082015181840152602081019050620001d5565b60008484015250505050565b6000620002156200020f846200019c565b6200017d565b90508281526020810184848401111562000234576200023362000102565b5b62000241848285620001d2565b509392505050565b600082601f830112620002615762000260620000fd565b5b815162000273848260208601620001fe565b91505092915050565b600080600060608486031215620002985762000297620000b8565b5b6000620002a886828701620000e6565b935050602084015167ffffffffffffffff811115620002cc57620002cb620000bd565b5b620002da8682870162000249565b925050604084015167ffffffffffffffff811115620002fe57620002fd620000bd565b5b6200030c8682870162000249565b9150509250925092565b600081519050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b600060028204905060018216806200036957607f821691505b6020821081036200037f576200037e62000321565b5b50919050565b60008190508160005260206000209050919050565b60006020601f8301049050919050565b600082821b905092915050565b600060088302620003e97fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff82620003aa565b620003f58683620003aa565b95508019841693508086168417925050509392505050565b6000819050919050565b600062000438620004326200042c84620000c2565b6200040d565b620000c2565b9050919050565b6000819050919050565b620004548362000417565b6200046c62000463826200043f565b848454620003b7565b825550505050565b600090565b6200048362000474565b6200049081848462000449565b505050565b5b81811015620004b857620004ac60008262000479565b60018101905062000496565b5050565b601f8211156200050757620004d18162000385565b620004dc846200039a565b81016020851015620004ec578190505b62000504620004fb856200039a565b83018262000495565b50505b505050565b600082821c905092915050565b60006200052c600019846008026200050c565b1980831691505092915050565b600062000547838362000519565b9150826002028217905092915050565b620005628262000316565b67ffffffffffffffff8111156200057e576200057d62000118565b5b6200058a825462000350565b62000597828285620004bc565b600060209050601f831160018114620005cf5760008415620005ba578287015190505b620005c6858262000539565b86555062000636565b601f198416620005df8662000385565b60005b828110156200060957848901518255600182019150602085019450602081019050620005e2565b8683101562000629578489015162000625601f89168262000519565b8355505b6001600288020188555050505b505050505050565b610a71806200064e6000396000f3fe608060405234801561001057600080fd5b506004361061007d5760003560e01c8063313ce5671161005b578063313ce567146100ee57806370a082311461010c57806395d89b411461013c578063a9059cbb1461015a5761007d565b806306fdde031461008257806318160ddd146100a057806323b872dd146100be575b600080fd5b61008a61018a565b6040516100979190610732565b60405180910390f35b6100a8610218565b6040516100b5919061076d565b60405180910390f35b6100d860048036038101906100d39190610817565b61021e565b6040516100e59190610885565b60405180910390f35b6100f66103f3565b60405161010391906108bc565b60405180910390f35b610126600480360381019061012191906108d7565b6103f8565b604051610133919061076d565b60405180910390f35b610144610440565b6040516101519190610732565b60405180910390f35b610174600480360381019061016f9190610904565b6104ce565b6040516101819190610885565b60405180910390f35b6002805461019790610973565b80601f01602080910402602001604051908101604052809291908181526020018280546101c390610973565b80156102105780601f106101e557610100808354040283529160200191610210565b820191906000526020600020905b8154815290600101906020018083116101f357829003601f168201915b505050505081565b60015481565b6000816000808673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054101561026b57600080fd5b816000808673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020546102b591906109d3565b6000808673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002081905550816000808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020546103419190610a07565b6000808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020819055508273ffffffffffffffffffffffffffffffffffffffff168473ffffffffffffffffffffffffffffffffffffffff167fddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef846040516103e0919061076d565b60405180910390a3600190509392505050565b601281565b60008060008373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020549050919050565b6003805461044d90610973565b80601f016020809104026020016040519081016040528092919081815260200182805461047990610973565b80156104c65780601f1061049b576101008083540402835291602001916104c6565b820191906000526020600020905b8154815290600101906020018083116104a957829003601f168201915b505050505081565b6000816000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054101561051b57600080fd5b816000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205461056591906109d3565b6000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002081905550816000808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020546105f19190610a07565b6000808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020819055508273ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff167fddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef84604051610690919061076d565b60405180910390a36001905092915050565b600081519050919050565b600082825260208201905092915050565b60005b838110156106dc5780820151818401526020810190506106c1565b60008484015250505050565b6000601f19601f8301169050919050565b6000610704826106a2565b61070e81856106ad565b935061071e8185602086016106be565b610727816106e8565b840191505092915050565b6000602082019050818103600083015261074c81846106f9565b905092915050565b6000819050919050565b61076781610754565b82525050565b6000602082019050610782600083018461075e565b92915050565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b60006107b88261078d565b9050919050565b6107c8816107ad565b81146107d357600080fd5b50565b6000813590506107e5816107bf565b92915050565b6107f481610754565b81146107ff57600080fd5b50565b600081359050610811816107eb565b92915050565b6000806000606084860312156108305761082f610788565b5b600061083e868287016107d6565b935050602061084f868287016107d6565b925050604061086086828701610802565b9150509250925092565b60008115159050919050565b61087f8161086a565b82525050565b600060208201905061089a6000830184610876565b92915050565b600060ff82169050919050565b6108b6816108a0565b82525050565b60006020820190506108d160008301846108ad565b92915050565b6000602082840312156108ed576108ec610788565b5b60006108fb848285016107d6565b91505092915050565b6000806040838503121561091b5761091a610788565b5b6000610929858286016107d6565b925050602061093a85828601610802565b9150509250929050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b6000600282049050600182168061098b57607f821691505b60208210810361099e5761099d610944565b5b50919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052601160045260246000fd5b60006109de82610754565b91506109e983610754565b9250828203905081811115610a0157610a006109a4565b5b92915050565b6000610a1282610754565b9150610a1d83610754565b9250828201905080821115610a3557610a346109a4565b5b9291505056fea26469706673582212204e518193080241b4d73009104188dfa9ab8b28f8428df9cb84c38b347daa7b6264736f6c63430008120033'
abi = load_abi('ERC20_compData.json')
print(abi)
#create_contract_transaction(abi, byte, 3000)
#create_contract_transaction((f['abi']),f['']